![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# Callysto's Weekly Data Visualization


## Global Food Prices

### Recommended Grade levels: 9-12

### Instructions

Click "Cell" and select "Run All".

This will import the data and run all the code, so you can see this week's data visualization. Scroll back to the top after you’ve run the cells.

![instructions](https://github.com/callysto/data-viz-of-the-week/blob/main/images/instructions.png?raw=true)

**You don't need to do any coding to view the visualizations**.

The plots generated in this notebook are interactive. You can hover over and click on elements to see more information. 

Email contact@callysto.ca if you experience issues.

### About this Notebook

Callysto's Weekly Data Visualization is a learning resource that aims to develop data literacy skills. We provide Grades 5-12 teachers and students with a data visualization, like a graph, to interpret. This companion resource walks learners through how the data visualization is created and interpreted by a data scientist. 

The steps of the data analysis process are listed below and applied to each weekly topic.

1. Question - What are we trying to answer?
2. Gather - Find the data source(s) you will need. 
3. Organize - Arrange the data, so that you can easily explore it. 
4. Explore - Examine the data to look for evidence to answer the question. This includes creating visualizations. 
5. Interpret - Describe what's happening in the data visualization. 
6. Communicate - Explain how the evidence answers the question. 

## Question

What are the prices of food around the world, and what can we learn from fluctuations in the prices of food globally? 

### Goal

Our mission is to look at how much common foods cost in different parts of the world and over different times. Specifically, we want to compare food prices for certain food items across countries to examine which countries have high and low prices.

### Background

Analyzing food item prices on a global scale holds significant importancce due to its far-reaching implications for individuals, communities, and economies. According to [this](https://cdn.dal.ca/content/dam/dalhousie/pdf/sites/agri-food/Canada%27s%20Food%20Price%20Report%202023_Digital.pdf) report on Canada's food prices, total foods increased in price by 10.3%.  Access to affordable and nutritious food is a fundamental human right, and understanding the dynamics of food prices is essential in addressing global challenges such as hunger and poverty. 

## Gather

Global food price data was collected through [Kaggle](https://www.kaggle.com/datasets/lasaljaywardena/global-food-prices-dataset?resource=download), which is sourced by the [WFP](https://www.wfp.org/) (The World Food Program). The dataset is distributed by [HDX](https://data.humdata.org/). 

An important thing to note in this dataset is that food prices are relative to each food item's respective country. This means that food prices are not normalized. For example, if you're looking at the prices of different food items in different countries, these prices might be influenced by various factors such as local economies, currency exchange rates, etc. Normalization could involve converting all prices to a common currency or adjusting them based on a common economic indicator, like GDP per capita, to make the comparisons more reflective of their actual cost.

### Code: 

Run the code cells below to import the libraries we need for this project. Libraries are pre-made code that make it easier to analyze our data.

In [22]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
print("Libaries imported")

Libaries imported


### Data

To begin, we're going to obtain our dataset by importing it using *pd.read_csv*. We're also going to ensure that when reading our data we enforce the column `adm1_name` to being only string values. A string, in the context of computing, is a sequence of characters, such as letters, numbers, symbols, and spaces, that is used to represent text-based information.  


We establish this practice because, without this constraint, Python (our chosen programming language) would alert us to columns containing multiple data types upon data import. Ensuring a consistent data type for each column in Python offers a range of advantages. It guarantees data precision and integrity by averting irregularities when working with columns, ultimately enhancing the reliability of our data utilization.

### Import the data

In [23]:
data_types = {'adm1_name': str}
global_food_prices= pd.read_csv('https://swift-yeg.cloud.cybera.ca:8080/v1/AUTH_233e84cd313945c992b4b585f7b9125d/callysto-open-data/global-food-prices.csv', dtype=data_types)
global_food_prices

### Comment on the data

Now that we've obtained our data, let's take a deeper look into the data types for each column. Note, our dataset is now in a dataframe format. Essentially, think of a dataframe as a spreadsheet where we have columns representing different categories or attributes of the data, and rows representing individual records or observations. Just like a spreadsheet organizes information into rows and columns, a dataframe provides a structured format for storing and analyzing data in Python.

In [ ]:
print(global_food_prices.dtypes)

adm0_name              object
adm1_name              object
mkt_name               object
cm_name                object
cur_name               object
pt_name                object
um_name                object
mp_month                int64
mp_year                 int64
mp_price              float64
mp_commoditysource    float64
dtype: object


Most of these datatypes make sense, such as `mp_month` and `mp_year` being integers (numbers) and `mp_price` and `mp_commoditysource` being floats (decimal point numbers). 

Let's begin by doing some basic analysis. Let's obtain the frequency of occurences of unique values in the `pt_name` column. In other words, it quantifies how often each distinct value appears in the specified column.

In [ ]:
global_food_prices['pt_name'].value_counts()

Retail       1287998
Wholesale     150711
Farm Gate        664
Producer         248
Name: pt_name, dtype: int64

The output reveals that the category *Retail* appears the most frequently, with a count of 1,287,998 occurrences. The category *Wholesale* follows with 150,711 occurrences, while *Farm Gate* and *Producer* categories are less prevalent, having counts of 664 and 248, respectively. Globally, it appears that the majority of the world utilizes retail items, which makes sense as shopping stores have become accessible to many countries and are the primary way many individuals obtain food items in the modern-day. 

# Organize

Let's begin to organize our data. Generally, when we organize data we perform something called *data-cleaning*. Data cleaning, an essential step in the data preparation process, involves identifying, correcting, and handling errors, inconsistencies, and inaccuracies within a dataset. It encompasses tasks such as removing duplicate or irrelevant entries, filling in missing values, correcting inaccuracies, standardizing formats, and ensuring uniformity in data representation. The goal of data cleaning is to enhance data quality, reliability, and usability, enabling accurate analyses and insights. 

To begin our data-cleaning, let's check if any of columns have *null* values (missing values).

In [ ]:
global_food_prices.isna().sum()

adm0_name                   0
adm1_name                   0
mkt_name                    0
cm_name                     0
cur_name                    0
pt_name                     0
um_name                     0
mp_month                    0
mp_year                     0
mp_price                    0
mp_commoditysource    1439621
dtype: int64

It appears that the only column that has *null* values is `mp_commoditysource`, at a staggering 1439621. It appears that there are too many *null* values for this column to be of any importance, so let's simply drop the column.

In [ ]:
global_food_prices = global_food_prices.drop('mp_commoditysource', axis=1)
global_food_prices

,adm0_name,adm1_name,mkt_name,cm_name,cur_name,pt_name,um_name,mp_month,mp_year,mp_price
0,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,1,2014,50.0000
1,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,2,2014,50.0000
2,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,3,2014,50.0000
3,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,4,2014,50.0000
4,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,5,2014,50.0000
...,...,...,...,...,...,...,...,...,...,...
1439616,Zimbabwe,Midlands,Mbilashaba,Salt - Retail,ZWL,Retail,KG,6,2021,71.0000
1439617,Zimbabwe,Midlands,Mbilashaba,Beans (sugar) - Retail,ZWL,Retail,KG,6,2021,233.3333
1439618,Zimbabwe,Midlands,Mbilashaba,Toothpaste - Retail,ZWL,Retail,100 ML,6,2021,112.5000
1439619,Zimbabwe,Midlands,Mbilashaba,Laundry soap - Retail,ZWL,Retail,KG,6,2021,114.0000


In [ ]:
global_food_prices

,adm0_name,adm1_name,mkt_name,cm_name,cur_name,pt_name,um_name,mp_month,mp_year,mp_price
0,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,1,2014,50.0000
1,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,2,2014,50.0000
2,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,3,2014,50.0000
3,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,4,2014,50.0000
4,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,5,2014,50.0000
...,...,...,...,...,...,...,...,...,...,...
1439616,Zimbabwe,Midlands,Mbilashaba,Salt - Retail,ZWL,Retail,KG,6,2021,71.0000
1439617,Zimbabwe,Midlands,Mbilashaba,Beans (sugar) - Retail,ZWL,Retail,KG,6,2021,233.3333
1439618,Zimbabwe,Midlands,Mbilashaba,Toothpaste - Retail,ZWL,Retail,100 ML,6,2021,112.5000
1439619,Zimbabwe,Midlands,Mbilashaba,Laundry soap - Retail,ZWL,Retail,KG,6,2021,114.0000


Lastly, let's change some column names to reflect their data more accurately. Specifically, we'll change the names of the columns `adm0_name` and `adm1_name` to `country` and `region`, respectively. This way, the column names will better match the kind of information they contain.

In [ ]:
global_food_prices.rename(columns={'adm0_name': 'country', 'adm1_name': 'region'}, inplace=True)
global_food_prices

,country,region,mkt_name,cm_name,cur_name,pt_name,um_name,mp_month,mp_year,mp_price
0,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,1,2014,50.0000
1,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,2,2014,50.0000
2,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,3,2014,50.0000
3,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,4,2014,50.0000
4,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,5,2014,50.0000
...,...,...,...,...,...,...,...,...,...,...
1439616,Zimbabwe,Midlands,Mbilashaba,Salt - Retail,ZWL,Retail,KG,6,2021,71.0000
1439617,Zimbabwe,Midlands,Mbilashaba,Beans (sugar) - Retail,ZWL,Retail,KG,6,2021,233.3333
1439618,Zimbabwe,Midlands,Mbilashaba,Toothpaste - Retail,ZWL,Retail,100 ML,6,2021,112.5000
1439619,Zimbabwe,Midlands,Mbilashaba,Laundry soap - Retail,ZWL,Retail,KG,6,2021,114.0000


# Explore

Now that we've performed some basic data-cleaning, let's begin some data exploration how visualizing the top 20 most frequent foods in our dataset. *Frequency* is defined as a count of categorized data, but it is not the actual values of the data points.

In [ ]:
count_of_food = global_food_prices['cm_name'].value_counts().nlargest(20)
top_20 = pd.DataFrame({'cm_name': count_of_food.index, 'count': count_of_food.values})

top_20_foods = px.bar(top_20, x='cm_name', y='count', title='Top 20 Foods by Highest Count', color='count', labels={"count":"Count", "cm_name":"Foods"})
top_20_foods.show()

The visualization titled *Top 20 Foods by Highest Count* highlights that the prevailing items with the highest frequency are Millet - Retail, Rice (imported) - Retail, and Maize - Retail. Remarkably, all of these food items fall under the Retail category. This observation aligns with our earlier discovery that Retail items dominate our dataset in terms of prevalence. Such a pattern offers insights into the consumer-oriented nature of the data, suggesting that retail-level transactions are the main source of food transactions, possibly reflecting consumption trends and preferences.

Let's determine the highest and lowest years represented in our dataframe. We'll want this information for our following visualization after this cell.

In [ ]:
highest_year = global_food_prices['mp_year'].max()
lowest_year = global_food_prices['mp_year'].min()
print(f"Highest Year: {highest_year}")
print(f"Lowest Year: {lowest_year}")

Highest Year: 2021
Lowest Year: 1990


With the highest and lowest years now identified in our dataframe, let's proceed to create a comparable visualization as previously done. This visualization will highlight the top 20 food items for both the highest and lowest years, or in other words, finding the top 20 food items with the highest frequencies in the dataset.  

In [ ]:
high_temp = global_food_prices[global_food_prices['mp_year'] == highest_year]
low_temp = global_food_prices[global_food_prices['mp_year'] == lowest_year]

top_20_foods_highest_year = high_temp['cm_name'].value_counts().nlargest(20)
top_20_highest_year = pd.DataFrame({'cm_name': top_20_foods_highest_year.index, 'count': top_20_foods_highest_year.values})

top_20_foods_lowest_year = low_temp['cm_name'].value_counts().nlargest(20)
top_20_lowest_year = pd.DataFrame({'cm_name': top_20_foods_lowest_year.index, 'count': top_20_foods_lowest_year.values})

high_low_fig = make_subplots(rows=2, cols=1, vertical_spacing=0.2)

high_low_fig.add_trace(go.Bar(x=top_20_highest_year['cm_name'], y=top_20_highest_year['count'], name='Highest Year'), row=1, col=1)
high_low_fig.add_trace(go.Bar(x=top_20_lowest_year['cm_name'], y=top_20_lowest_year['count'], name='Lowest Year'), row=2, col=1)

high_low_fig.update_layout(title_text=f"Top 20 Foods by Highest Count in Highest ({highest_year}) and Lowest ({lowest_year}) Years")
high_low_fig.update_yaxes(title="Count", row=1, col=1)
high_low_fig.update_yaxes(title="Count", row=2, col=1)


high_low_fig.update_layout(height=800)

high_low_fig.show()

Starting with 2021, it appears that certain items such as *Sugar*, *Salt*, and *Oil* dominate the frequency distribution. Sugar having the highest frequency, would be defined as the *mode*, a mathematical term defining the value that appears most frequently in a given set of data. This observation aligns with expectations, given that these items often serve as foundational staples in various cuisines and diets, making them widely sought after and widely available. 

Examining the data for the year 1990, a stark contrast emerges as we find only three items represented: *Millet*, *Sorghum*, and *Rice*. This limited diversity could be attributed to several factors. One potential factor is that data collection practices and record-keeping back in the early 1990s might have been less comprehensive compared to contemporary times, resulting in a narrower range of documented food items. 

Notice how the *context* of data greatly impacts what is considered a high frequency or low frequency food item. In the year 1990, millet is considered the most frequent (mode) item in the given dataset with a count of 58. However, if we put this item into the context of the year 2021, it would be considered one of the least frequent food items. This shows us that when dealing with data looking at numbers alone doesn't show the full story - looking at how the numbers fit relative to the data helps us make sense of their significance.

We noticed that Millet was a common food item between the 2021 and 1990 figures. Let's filter our data on this particular food item to compare the differences in prices. To do this, we're going to filter the data on our specified year and food item. Then, we're going to retrieve the maximum price for each country as each country can have multiple prices of the same item. 

Similarly to before, by comparing the prices between different times, we get to see if the item has generally increased or decreased in price. 

In [ ]:
# Filter for 'Millet - Retail' prices and the year 2021
millet_retail_data_2021 = global_food_prices[
    (global_food_prices['cm_name'] == 'Millet - Retail') &
    (global_food_prices['mp_year'] == 2021)
]

# We're going to getting only the highest price for each country 
indices_highest_prices = millet_retail_data_2021.groupby('country')['mp_price'].idxmax()
millet_retail_highest_prices_2021 = millet_retail_data_2021.loc[indices_highest_prices]
millet_retail_highest_prices_2021 = millet_retail_highest_prices_2021.sort_values(by='mp_price', ascending=False)

millet_plots = px.scatter(millet_retail_highest_prices_2021, x='country',y='mp_price',color='mp_price',title='Highest Retail Millet Prices in 2021',hover_data=['cur_name', 'mp_month'], labels={"country":"Country", "mp_price":"Price"}).show()

Now let's filter for the years 1990-2000. The reason why we don't only filter on 1990 is because there are so few datapoints in the year 1990 alone. This broader time frame takes into account the relative scarcity of data points in the solitary year of 1990, ensuring a more comprehensive representation of trends and fluctuations.

In [ ]:
# Filter for 'Millet - Retail' prices and years 1990-2000
millet_retail_data_2000 = global_food_prices[
    (global_food_prices['cm_name'] == 'Millet - Retail') &
    (global_food_prices['mp_year'] <= 2000)
]

# We're going to getting only the highest price for each country 
indices_highest_prices = millet_retail_data_2000.groupby('country')['mp_price'].idxmax()
millet_retail_highest_prices_2000 = millet_retail_data_2000.loc[indices_highest_prices]
millet_retail_highest_prices_2000 = millet_retail_highest_prices_2000.sort_values(by='mp_price', ascending=False)

millet_plots = px.scatter(millet_retail_highest_prices_2000, x='country',y='mp_price',color='mp_price',title='Highest Retail Millet Prices from 1990-2000',hover_data=['cur_name', 'mp_month', 'mp_year'], labels={"country":"Country", "mp_price":"Price"}).show()

In [ ]:
price_differences = []

for ind in millet_retail_highest_prices_2000.index:
    country_name = millet_retail_highest_prices_2000.loc[ind, 'country']
    currency_name = millet_retail_highest_prices_2000.loc[ind, 'cur_name']
    
    if country_name in millet_retail_highest_prices_2021['country'].values:
        price_diff = millet_retail_highest_prices_2021.loc[millet_retail_highest_prices_2021['country'] == country_name, 'mp_price'].values[0] - millet_retail_highest_prices_2000.loc[ind, 'mp_price']
        price_differences.append((country_name, price_diff, currency_name))

print(price_differences)

[('Niger', 52.0, 'XOF'), ('Burkina Faso', 88.4801, 'XOF'), ('Senegal', 205.625, 'XOF')]


Comparing the two visualizations, we notice 3 common datapoints. Beginning with *Burkina Faso*, in 1998 the highest price of Millet was 246.5199 XOF dollars. In 2021, the price increased to 335 XOF dollars. For *Niger*, the highest price in 1998 was 312 XOF dollars. This price also increased in 2021, to 364 XOF dollars. Lastly, *Senegal*'s highest price in 2000 was 144.375 XOF. In 2021, this price also increased to 350 XOF dollars. Reflect on this output, what factors could contribute to the consistent price increase of Millet across these countries over the years?

In our final section of this notebook, we are going to be comparing the progression of prices of the food items in our dataset, alongside utilizing machine-learning to potentially predict the prices of items in future years. To begin, let's list all the countries in our dataset in order to filter our data under a single country's currency. Mixing different currencies would be bad practice as many countries utilize different currencies. 

In [ ]:
all_countries = global_food_prices['country'].unique()
print(f"Countries in the dataset: \n{all_countries}")

Countries in the dataset: 
['Afghanistan' 'Algeria' 'Angola' 'Argentina' 'Bangladesh' 'Belarus'
 'Benin' 'Bolivia' 'Burkina Faso' 'Burundi' 'Cambodia' 'Cameroon'
 'Cape Verde' 'Central African Republic' 'China' 'Colombia' 'Congo'
 "Cote d'Ivoire" 'Democratic Republic of the Congo' 'Djibouti' 'Ecuador'
 'Eritrea' 'Ethiopia' 'Gabon' 'Gambia' 'Georgia' 'Ghana' 'Guinea'
 'Guinea-Bissau' 'Haiti' 'Iran  (Islamic Republic of)' 'Japan'
 'Kazakhstan' 'Kenya' "Lao People's Democratic Republic" 'Lesotho'
 'Liberia' 'Madagascar' 'Malawi' 'Mali' 'Mauritania' 'Mexico'
 'Moldova Republic of' 'Mongolia' 'Myanmar' 'Namibia' 'Nepal' 'Niger'
 'Nigeria' 'Pakistan' 'Paraguay' 'Peru' 'Philippines' 'Russian Federation'
 'Rwanda' 'Senegal' 'Sierra Leone' 'Somalia' 'South Africa'
 'State of Palestine' 'Sudan' 'Syrian Arab Republic' 'Thailand'
 'Timor-Leste' 'Togo' 'Uganda' 'United Republic of Tanzania' 'Viet Nam'
 'Yemen' 'Zambia' 'Zimbabwe']


Now that we have a list of all the different countries in our dataframe, we can visualize different trends for the prices of food items in our dataframe. In our case, let's choose Rwanda, specifically as it's a country which contains many datapoints for the machine-learning portion later on. Let's begin by visualizing the trends of the highest and lowest prices every year based on the different markets in our particular country chosen.  

In [ ]:
user_country = 'Rwanda'

country_subset = global_food_prices[global_food_prices['country'] == user_country]

indices_highest_prices = country_subset.groupby(['mp_year', 'mkt_name'])['mp_price'].idxmax()
indices_lowest_prices = country_subset.groupby(['mp_year', 'mkt_name'])['mp_price'].idxmin()

highest_prices_per_year_market = country_subset.loc[indices_highest_prices].reset_index(drop=True)
lowest_prices_per_year_market = country_subset.loc[indices_lowest_prices].reset_index(drop=True)

pd.set_option('display.max_rows', 60)

print("Highest Prices per Year and Market:")
display(highest_prices_per_year_market)

print("\nLowest Prices per Year and Market:")
display(lowest_prices_per_year_market)

Highest Prices per Year and Market:


,country,region,mkt_name,cm_name,cur_name,pt_name,um_name,mp_month,mp_year,mp_price
0,Rwanda,Kigali City/Umujyi wa Kigali,Kigali,Beans - Wholesale,RWF,Wholesale,KG,11,2000,239.7290
1,Rwanda,Kigali City/Umujyi wa Kigali,Kigali,Maize - Wholesale,RWF,Wholesale,KG,1,2001,165.8820
2,Rwanda,Kigali City/Umujyi wa Kigali,Kigali,Beans - Wholesale,RWF,Wholesale,KG,12,2002,117.6600
3,Rwanda,Kigali City/Umujyi wa Kigali,Kigali,Beans - Wholesale,RWF,Wholesale,KG,11,2003,137.2240
4,Rwanda,Kigali City/Umujyi wa Kigali,Kigali,Beans - Wholesale,RWF,Wholesale,KG,6,2004,124.5490
...,...,...,...,...,...,...,...,...,...,...
616,Rwanda,East/Iburasirazuba,Nyabiheke (Camp),Oil - Retail,RWF,Retail,L,7,2021,2650.0000
617,Rwanda,East/Iburasirazuba,Nyakarambi,Meat (beef) - Retail,RWF,Retail,KG,1,2021,3500.0000
618,Rwanda,East/Iburasirazuba,Nyamugari,Oil - Retail,RWF,Retail,L,5,2021,2500.0000
619,Rwanda,North/Amajyaruguru,Ruhengeri,Rice - Wholesale,RWF,Wholesale,KG,6,2021,852.0690



Lowest Prices per Year and Market:


,country,region,mkt_name,cm_name,cur_name,pt_name,um_name,mp_month,mp_year,mp_price
0,Rwanda,Kigali City/Umujyi wa Kigali,Kigali,Beans - Wholesale,RWF,Wholesale,KG,1,2000,108.016
1,Rwanda,Kigali City/Umujyi wa Kigali,Kigali,Beans - Wholesale,RWF,Wholesale,KG,2,2001,127.305
2,Rwanda,Kigali City/Umujyi wa Kigali,Kigali,Maize - Wholesale,RWF,Wholesale,KG,2,2002,101.954
3,Rwanda,Kigali City/Umujyi wa Kigali,Kigali,Maize - Wholesale,RWF,Wholesale,KG,2,2003,63.377
4,Rwanda,Kigali City/Umujyi wa Kigali,Kigali,Maize - Wholesale,RWF,Wholesale,KG,4,2004,94.790
...,...,...,...,...,...,...,...,...,...,...
616,Rwanda,East/Iburasirazuba,Nyabiheke (Camp),Cabbage - Retail,RWF,Retail,KG,3,2021,100.000
617,Rwanda,East/Iburasirazuba,Nyakarambi,Cabbage - Retail,RWF,Retail,KG,4,2021,100.000
618,Rwanda,East/Iburasirazuba,Nyamugari,Cabbage - Retail,RWF,Retail,KG,5,2021,100.000
619,Rwanda,North/Amajyaruguru,Ruhengeri,Maize - Wholesale,RWF,Wholesale,KG,4,2021,203.863


Looking at the output, it appears that for the Rwandan market, the months of certain items heavily dictate the price of the item for the year. For example, *beans* and *maize* both appear as highest priced and lowest priced items. 

Now that we've visualized the different food prices for items in a particular market, let's move onto the final machine-learning portion of this notebook. In the context of analyzing global food prices, machine learning provides a powerful tool to uncover patterns and relationships within large datasets, offering insights into how various factors influence price trends. By leveraging algorithms like *Linear Regression*, it becomes possible to model and predict price changes based on historical data. This enables us to identify potential drivers behind price fluctuations.

To begin, we're going to list all the food items in our chosen country.

In [ ]:
# Set output to no limit
pd.set_option('display.max_rows', None)
country_subset['cm_name'].value_counts()

Beans (dry) - Retail                               5572
Maize flour - Retail                               5409
Potatoes (Irish) - Retail                          5399
Cassava flour - Retail                             5292
Maize - Retail                                     5165
Sorghum - Retail                                   5104
Bananas - Retail                                   4644
Sweet potatoes - Retail                            4422
Passion fruit - Retail                             4026
Cassava - Retail                                   3679
Eggplants - Retail                                 3624
Meat (beef) - Retail                               3237
Peppers (green) - Retail                           3171
Sorghum flour - Retail                             3133
Cabbage - Retail                                   2866
Charcoal - Retail                                  2799
Rice - Retail                                      2603
Papaya - Retail                                 

From the items above, we can select a particular item to create a model on. Generally, we want to choose items that have more data-points (the number on the right of the food item) as it serves as more fuel or "food" to send into our model to create accurate predictions.

In [ ]:
# If wanted, change the product from Beans (dry) - Retail to any product above
# Note: An product which has a larger value is much more likely to produce an ideal result for the model

product = 'Beans (dry) - Retail'
product_subset = country_subset[country_subset['cm_name'] == product]

X_train, X_test, y_train, y_test = train_test_split(product_subset['mp_price'], product_subset['mp_year'], test_size=0.2, random_state=15)

X_train = X_train.values.reshape(-1, 1)
X_test = X_test.values.reshape(-1, 1)
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print("R-squared (Training Set):", r2_train)
print("R-squared (Test Set):", r2_test)

R-squared (Training Set): 0.399241027335253
R-squared (Test Set): 0.4106266150345035


Based on our r-squared values outputted, we can get a better sense of how our models are doing. Generally, we want models to have a higher r-squared value from the range of 0 to 1, as it is indicative of the performance of the model. In our case, our test set has an r-squared value of 0.41, which is considered a low r-squared error value reflecting that our data is not necessarily reliable.

In [ ]:
scatter_test = px.scatter(x=X_test.flatten(), y=y_test.flatten(), title=f'Predicted Prices on {product}', labels={'x': 'Year', 'y': 'Price'})
scatter_test.add_trace(go.Scatter(x=X_test.flatten(), y=y_pred_test.flatten(), mode='lines', name='Regression Line'))
scatter_test.show()

Looking at the output above, it appears that a general upwards trend was found in our model. This makes sense with our current understandings of food inflation costs. However, with the various scattering of data-points on our model due to many outliers, it appears to be very difficult to generalize values to years. To reduce the scattering, let's just take the median-value of each year so that we have easier visualization to work with.

In [ ]:
median_prices_per_year = product_subset.groupby('mp_year')['mp_price'].median().reset_index()
median_prices_per_year

,mp_year,mp_price
0,2008,301.25000
1,2009,265.00000
2,2010,287.50000
3,2011,303.33330
4,2012,330.00000
5,2013,382.66665
6,2014,389.40000
7,2015,392.50000
8,2016,545.00000
9,2017,435.00000


In [ ]:
median_prices_per_year = product_subset.groupby('mp_year')['mp_price'].median().reset_index()

X = median_prices_per_year[['mp_year']]
y = median_prices_per_year['mp_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

median_model = LinearRegression()
median_model.fit(X_train, y_train)

y_pred = median_model.predict(X_test)

r2 = r2_score(y_test, y_pred)
print(f'R-squared value: {r2}')

prediction_df = pd.DataFrame({'mp_year': X_test['mp_year'], 'Actual Price': y_test, 'Predicted Price': y_pred})

median_model_plot = px.scatter(prediction_df, y=['Actual Price', 'Predicted Price'], x='mp_year',
                          title=f'Actual vs. Predicted Prices for {product}', labels={'value': 'Price', 'mp_year': 'Year'}).show()

R-squared value: 0.8256066173755379


Looking at the new model, it appears that our values calculated through the median costs appears to be working better. The biggest difference between *actual* and *predicted* values is in the year 2021, with the other years being fairly close to their predicted values. Nonetheless, it does appear that the model is still predicting values to be gradually increasing as years progress, indicating that without fluctuations between months, it is safe to assume prices to generally increase as years increase. 

However, take the data seen with a *grain of salt*. While it is generally true that prices increase as years progress, this isn't always true for all food items. As seen in the plot before, there are certain years when items decrease in price despite being a higher year. Therefore, at the end of the day, individuals have to be the ones to make smart, educated choices on the cost of food with respect to the information that is presented.

# Interpret

### Reflect on What You See

Think about the following questions.

1. How have rising food prices impacted your own consumption patterns and budget allocation for food?
2. How might increasing food prices speak about income inequality and access to nutritious diets?
3. What lessons can be drawn from historical instances of food price spikes, and how can they inform proactive measures to address future challenges related to increasing food costs?

# Communicate

Below are some writing prompts to help you reflect on the new information that is presented from the data. When we look at the evidence, think about what you perceive about the information. Is this perception based on what the evidence shows? If others were to view it, what perceptions might they have?

- I used to think ____________________ but now I know ____________________. 
- I wish I knew more about ____________________. 
- This visualization reminds me of ____________________. 
- I really like ____________________.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)